In [3]:
import sys
import torch
sys.path.append("..")
import logging
from random import seed
import neptune
import numpy as np
import syft as sy
from copy import deepcopy
from collections import defaultdict
from torch import load
from random import sample
from torchvision import transforms
from federated_learning.FLCustomDataset import FLCustomDataset
from federated_learning.FederatedLearning import FederatedLearning
from federated_learning.helper import utils

arguments = dict()
arguments['--output-prefix'] = ""

CONFIG_PATH = '../configs/defaults.yml'

configs = utils.load_config(CONFIG_PATH)
logging.basicConfig(format='%(asctime)s %(message)s', level=configs['log']['level'])
random_seed = configs['runtime']['random_seed']
seed(random_seed)

# Logging initialization
log_enable = False
output_dir = None
if log_enable:
    output_dir = utils.make_output_dir(
        configs['log']['root_output_dir'], arguments['--output-prefix'])
    utils.save_configs(output_dir, configs)
neptune_enable = False

# Neptune logging initialization
if neptune_enable:
    neptune.init(configs['log']['neptune_init'])
    neptune.create_experiment(name = configs['log']['neptune_exp'])

# syft initialization
hook = sy.TorchHook(torch)
use_cuda = False
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
device = torch.device("cuda" if use_cuda else "cpu")
torch.manual_seed(random_seed)


In [5]:
from federated_learning.FLNet import FLNet
tmp_model = FLNet().to(torch.device("cpu"))

In [39]:
dd = tmp_model.state_dict()
for i, j in tmp_model.named_parameters():
    print(i)

print_model(tmp_model)

conv1.weight
conv1.bias
conv2.weight
conv2.bias
fc1.weight
fc1.bias
fc2.weight
fc2.bias
Parameter containing:
tensor([ 0.0603, -0.0477, -0.0789, -0.0583,  0.1467,  0.1109,  0.0871, -0.0192,
         0.0964,  0.0262, -0.0770, -0.0780,  0.0882,  0.0009,  0.1538,  0.1351,
         0.1136, -0.1533, -0.1837,  0.0886], requires_grad=True)


In [41]:
def print_model(model):
    for ii, jj in model.named_parameters():
        if ii == "conv1.bias":
            print(jj.data[:7])
            
print_model(tmp_model)

tensor([ 0.0603, -0.0477, -0.0789, -0.0583,  0.1467,  0.1109,  0.0871])


In [28]:
aa = torch.tensor([0] * torch.numel(bb)).view(bb.shape)
print(aa.shape)
# torch.reshape(aa, list(bb.size()))
# print(aa.shape)
print(bb.shape)

torch.Size([20, 1, 5, 5])
torch.Size([20, 1, 5, 5])


In [ ]:
from random import sample

selected_users_num = configs['mnist']['selected_users_num']
for round_no in range(rounds_num):
    # select selected_users_num users randomly
    workers_to_be_used = sample(workers_idx, selected_users_num)
    logging.info("Selected users for this round: {}".format(workers_to_be_used))

    logging.info("Create workers model for this round based on the server model...")
    workers_model = dict()
    for worker_id in workers_to_be_used:
        workers_model[worker_id] = deepcopy(fl.server_model)

    fl.train_workers(fed_train_dataloaders, workers_model, round_no, epochs_num)

    # Find the best weights and update the server model
    weights = None
    if arguments['--avg']:
        # Each worker takes two shards of 300 samples. Total of 600 samples
        # per worker. Total number of samples is 60000.
        weights = [600.0 / 60000] * selected_users_num
    # elif arguments['--opt']:
    #     # weights = fl.find_best_weights(trained_server_model, workers_idx)
    #     weights = fl.find_best_weights(trained_w0_model, workers_idx)

    if log_enable:
        fl.save_workers_model(workers_idx, str(round_no))
        # fl.save_model(
        #     fl.get_average_model(trusted_idx),
        #     "R{}_{}".format(round_no, "avg_trusted_model")
        # )

    logging.info("Update server model in this round...")
    fl.update_models(["server"], fl.wieghted_avg_model(weights, workers_idx))

    # Apply the server model to the test dataset
    fl.test(fl.server_model, test_dataloader, round_no)

    if log_enable:
        fl.save_model(
            fl.server_model, 
            "R{}_{}".format(round_no, "server_model")
        )

    print("")